In [52]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import easyocr
import imutils


In [ ]:
# Load the video
video_path = "TestVideo.mp4"  # Replace with your video file path
cap = cv2.VideoCapture(video_path)

# Check if the video file is loaded successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get frame rate to slow down playback
fps = cap.get(cv2.CAP_PROP_FPS)
delay = int(1000 / fps)

reader = easyocr.Reader(['en'])

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Bilateral filter to reduce noise while keeping edges sharp
    bfilter = cv2.bilateralFilter(gray, 11, 17, 17)

    # Canny edge detection
    edged = cv2.Canny(bfilter, 30, 200)

    # Find contours
    cnts = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(cnts)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

    location = None
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 10, True)
        if len(approx) == 4:
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = w / float(h)
            area = cv2.contourArea(contour)
            if 2 < aspect_ratio < 6 and area > 500:
                location = approx
                break

    if location is not None:
        mask = np.zeros(gray.shape, dtype=np.uint8)
        cv2.drawContours(mask, [location], -1, 255, -1)
        (x, y) = np.where(mask == 255)
        (x1, y1) = (np.min(x), np.min(y))
        (x2, y2) = (np.max(x), np.max(y))
        cropped_image = gray[x1:x2+1, y1:y2+1]

        # Resize for OCR
        cropped_image = cv2.resize(cropped_image, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)

        result = reader.readtext(cropped_image)
        if result:
            text = result[0][-2]
            font = cv2.FONT_HERSHEY_SIMPLEX
            frame = cv2.putText(frame, text=text, org=(location[0][0][0], location[1][0][1]+60),
                                fontFace=font, fontScale=1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
            frame = cv2.polylines(frame, [location], True, (0, 255, 0), 3)

    cv2.imshow("Video Processing", frame)

    # Control playback speed
    if cv2.waitKey(delay) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
